In [1]:
import re
import pandas as pd
import numpy as np
import dataframe
from bs4 import BeautifulSoup
import socket 
import time
import requests
from selenium import webdriver
from urllib.parse import quote_plus
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import urllib.request
import urllib.parse

In [10]:
ceo_df = pd.read_csv(r'C:\Users\Jeeho\Documents\Python Scripts\Capstone project\Web_Scrap\fg500_CEOs_2018_v2(Apr_25_2020).csv')

In [11]:
c_sample = ceo_df['CEO']

In [12]:
browser = webdriver.Chrome(r"C:\Users\Jeeho\AppData\Local\Programs\Python\Python36\chromedriver.exe")


ceo_list = c_sample
values =[]
df = pd.DataFrame(columns = ['ceo'])

for ceo in ceo_list:
    baseUrl = 'https://www.google.com/search?q='
    browser.get(baseUrl + ceo)
    table = browser.find_elements_by_css_selector('div.ifM9O') 

    for row in table:
        ceo = str(([c.text for c in row.find_elements_by_css_selector('div.kno-ecr-pt.PZPZlf.gsmt.i8lZMc')])).strip('[]').strip("''")
    for row in table:
        value = str(([c.text for c in row.find_elements_by_css_selector('div.Z1hOCe')])).strip('[]').strip("''")
    if not table:
        value = 'Null'
    
    values.append(value)
    
    s = pd.Series(ceo,index=df.columns)
    vs = pd.Series(values) 

    df = df.append(s,ignore_index=True)

    time.sleep(2) # To make the scraping slower
    
    
df['value'] = vs  
df['ceo_year']=2018
print(df)

browser.close()

                                             ceo  \
0                                  Doug McMillon   
1                                        Kou Wei   
2                                   Dai Houliang   
3                                  Zhang Jianhua   
4                                Ben van Beurden   
5                                    Akio Toyoda   
6                                  Herbert Diess   
7                                     Bob Dudley   
8                                   Darren Woods   
9                                 Warren Buffett   
10                                      Tim Cook   
11                                                 
12                            John H. Hammergren   
13                               Ivan Glasenberg   
14                                                 
15                                Dieter Zetsche   
16                                Larry J. Merlo   
17                                    Jeff Bezos   
18          

In [13]:
pd.set_option('display.max_rows', None)

In [14]:
#Make a copy of the original scraped dataset
df1 = df

In [15]:
#reindex the data
re_index = pd.Series(range(1,501))  
df1 = df1.set_index([re_index])

Rename the unnamed column as index

In [16]:
# convert the index to a column and name as company_rank 
df1['index1'] = df1.index

df1 = df1.rename(columns={'index1': 'company_rank'})

In [18]:
#reindex the data as starting from 1 to 500
re_index = pd.Series(range(1,501))  
df1 = df1.set_index([re_index])

Clear rows. Select the rows having values only

In [19]:
#Select only the rows with values to proceed RegEx later
# exclude the rows with ceo value having ''
df_n = df1[df1['ceo'] != '']

#exlucde the rows if the value of 'value' column is null
df_nv = df_n[(df_n['value'] != 'Null')]

In [20]:
# Exclude the rows if the value of 'company_rank' is null
df_nv = df_nv[(df_nv['company_rank'].notnull())]

In [21]:
# For Born
df_nv['Born'] = df_nv['value'].str.extract(r"Born: (.+?)\'\," )

# For Nationality 
df_nv['Nationality'] = df_nv['value'].str.extract(r"Nationality: (.+?)\'\," )


# For Education
## Select the strings end with "', '" and the strings end with nothing
Education = df_nv['value'].str.extract(r"Education: (.+?)\'\,\s\'|Education: (.+)?|Alma mater: (.+?)\'\,\s\'")

## merge the extracted two columes
df_nv['Education'] = pd.concat([Education[0].dropna(), Education[1].dropna(), Education[2].dropna()]).reindex_like(Education)

#See the full contents of the rows of value
pd.set_option('display.max_colwidth', -1)


Extract only Birth of Year only as BoY

In [22]:
df_nv['BoY'] = df_nv['Born'].str.extract(pat = "([0-9]{4})" )

Extract Nationality strings from Born based on 1) the postal abbreviations of the US and 2) the country names

In [23]:
# Create a list of the abbr

us_postal_abbr = {'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
                  'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
                  'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
                  'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
                  'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WY'}

In [24]:
# Create a column for the extracted us abbreviation code 
df_nv['us_abbr'] = df_nv['Born'].str.extract(pat = "([A-Z]{2})")

# Insert value 'American' in the 'Nationality' column if where the us_abbr matchs with the list of abbr
df_nv.loc[df_nv['us_abbr'].isin(us_postal_abbr), 'Nationality'] = 'American'

For nationalities

Catch the missing countries among the values

In [26]:
#Extract the United States and United Kingdom and the other country names
bn = df_nv['Born'].str.extract(".*?([A-Za-z]{6}\s[A-Za-z]{6,7})$|.*?([A-Za-z]{5,11})$")

## merge the extracted two columes
df_nv['born_country'] = pd.concat([bn[0].dropna(), bn[1].dropna()]).reindex_like(bn)

#Replacing the NaN values of Nationality column with newly extracted values 
df_nv['Nationality'].fillna(df_nv['born_country'], inplace=True)

In [27]:
#drop value, us_abbr, and born_coutry columns
df_nv = df_nv.drop(['value', 'us_abbr', 'born_country'], axis = 1)

In [28]:
df_nv1 = df_nv

In [29]:
# change nationality values into country values

# Nationality list
nat = ['Afghan', 'Albanian', 'Algerian', 'American', 'Andorran', 'Angolan', 'Anguillan', 'Argentine', 'Armenian', 
       'Australian', 'Austrian', 'Azerbaijani', 'Bahamian', 'Bahraini', 'Bangladeshi', 'Barbadian', 'Belarusian',
       'Belgian', 'Belizean', 'Beninese', 'Bermudian', 'Bhutanese', 'Bolivian', 'Botswanan', 'Brazilian', 'British',
       'British Virgin Islander', 'Bruneian', 'Bulgarian', 'Burkinan', 'Burmese', 'Burundian', 'Cambodian', 'Cameroonian',
       'Canadian', 'Cape Verdean', 'Cayman Islander', 'Central African', 'Chadian', 'Chilean', 'Chinese', 
       'Citizen of Antigua and Barbuda', 'Citizen of Bosnia and Herzegovina', 'Citizen of Guinea-Bissau', 
       'Citizen of Kiribati', 'Citizen of Seychelles', 'Citizen of the Dominican Republic', 'Citizen of Vanuatu ', 
       'Colombian', 'Comoran','Congolese (Congo)', 'Congolese (DRC)', 'Cook Islander', 'Costa Rican', 'Croatian', 'Cuban',
       'Cymraes', 'Cymro', 'Cypriot', 'Czech', 'Danish', 'Djiboutian', 'Dominican', 'Dutch', 'East Timorese', 'Ecuadorean',
       'Egyptian', 'Emirati', 'English', 'Equatorial Guinean', 'Eritrean', 'Estonian', 'Ethiopian', 'Faroese', 'Fijian',
       'Filipino', 'Finnish', 'French', 'Gabonese', 'Gambian', 'Georgian', 'German', 'Ghanaian', 'Gibraltarian', 'Greek',
       'Greenlandic', 'Grenadian', 'Guamanian', 'Guatemalan', 'Guinean', 'Guyanese', 'Haitian', 'Honduran', 'Hong Konger',
       'Hungarian', 'Icelandic', 'Indian', 'Indonesian', 'Iranian', 'Iraqi', 'Irish', 'Israeli', 'Italian', 'Ivorian', 
       'Jamaican', 'Japanese', 'Jordanian', 'Kazakh', 'Kenyan', 'Kittitian', 'Kosovan', 'Kuwaiti', 'Kyrgyz', 'Lao', 'Latvian',
       'Lebanese', 'Liberian', 'Libyan', 'Liechtenstein citizen', 'Lithuanian', 'Luxembourger', 'Macanese', 'Macedonian',
       'Malagasy', 'Malawian', 'Malaysian', 'Maldivian', 'Malian', 'Maltese', 'Marshallese', 'Martiniquais', 'Mauritanian',
       'Mauritian', 'Mexican', 'Micronesian', 'Moldovan', 'Monegasque', 'Mongolian', 'Montenegrin', 'Montserratian',
       'Moroccan', 'Mosotho', 'Mozambican', 'Namibian', 'Nauruan', 'Nepalese', 'New Zealander', 'Nicaraguan', 'Nigerian',
       'Nigerien', 'Niuean', 'North Korean', 'Northern Irish', 'Norwegian', 'Omani', 'Pakistani', 'Palauan', 'Palestinian',
       'Panamanian', 'Papua New Guinean', 'Paraguayan', 'Peruvian', 'Pitcairn Islander', 'Polish', 'Portuguese', 'Prydeinig',
       'Puerto Rican', 'Qatari', 'Romanian', 'Russian', 'Rwandan', 'Salvadorean', 'Sammarinese', 'Samoan', 'Sao Tomean',
       'Saudi Arabian', 'Scottish', 'Senegalese', 'Serbian', 'Sierra Leonean', 'Singaporean', 'Slovak', 'Slovenian',
       'Solomon Islander', 'Somali', 'South African', 'South Korean', 'South Sudanese', 'Spanish', 'Sri Lankan',
       'St Helenian', 'St Lucian', 'Stateless', 'Sudanese', 'Surinamese', 'Swazi', 'Swedish', 'Swiss', 'Syrian', 'Taiwanese',
       'Tajik', 'Tanzanian', 'Thai', 'Togolese', 'Tongan', 'Trinidadian', 'Tristanian', 'Tunisian', 'Turkish', 'Turkmen',
       'Turks and Caicos Islander', 'Tuvaluan', 'Ugandan', 'Ukrainian', 'Uruguayan', 'Uzbek', 'Vatican citizen', 'Venezuelan',
       'Vietnamese', 'Vincentian', 'Wallisian', 'Welsh', 'Yemeni', 'Zambian', 'Zimbabwean']

In [30]:
# create a value column "Check" showing Y if the value in country column is in the list of nationalities
df_nv1.loc[df_nv1['Nationality'].isin(nat), 'Check'] = 'Y'

In [31]:
''' a list of Nationality and other different country names among the Country values and correction
American = United States
Argentine = Argentina
Austrian = Austria
Brazilian = Brazil
British = United Kingdom
Canadian = Canada
Canada (English-speaking) = Canada
Canada French = Canada
Chinese = China
Danish = Denmark
French = France
German = Germany
Germany (EAST) = Germany
Germany (WEST) = Germany
Germany East = Germany
Indian = India
Japanese = Japan
Portuguese = Portugal
Singaporean = Singapore
Swedish = Sweden
Swiss = Switzerland
U.S. = United States
USA = United States
American, Indian = Indian
Korea = South Korea
Russian, German = Russia
French, Moroccan = Morocco
'''

a = ['American', 'Argentine', 'Austrian', 'Brazilian', 'British', 'Canadian', 'Canada (English-speaking)', 'Canada French', 
     'Chinese', 'Danish', 'French', 'German', 'Germany (EAST)', 'Germany (WEST)', 'Germany East', 'Indian', 'Japanese', 
     'Portuguese', 'Singaporean', 'Swedish', 'Swiss', 'U.S.', 'USA', "American, Indian", 'Korea', "Russian, German", "French, Moroccan"]
b = ['United States', 'Argentina', 'Austria', 'Brazil', 'United Kingdom', 'Canada', 'Canada', 'Canada', 'China', 'Denmark', 
     'France', 'Germany', 'Germany', 'Germany', 'Germany', 'India', 'Japan', 'Portugal', 'Singapore', 'Sweden', 'Switzerland',
     'United States', 'United States', 'India', 'South Korea', 'Russia', 'Morocco']

In [32]:
# Create a dataframe of the identified nationalities and thier matching countries
nat_correct = pd.DataFrame(columns = ('nat','correct'))

# Insert the list of nationalites and country names
nat_correct['nat'] = a
nat_correct['correct'] = b 

In [33]:
#Match the identified country names with nationalities in the country_v1 dataframe
df_nv1= nat_correct.set_index(['nat'])\
         .combine_first(df_nv1.set_index(['Nationality']))\
         .reset_index()
#Drop Nationality and correction columns


In [34]:
df_nv1['Nationality'] = np.where(df_nv1['correct'].notnull(), df_nv1['correct'], df_nv1['index'])

#Drop index, correct, and Check columns
df_nv1 = df_nv1.drop(['index', 'correct', 'Check'], axis = 1)

In [35]:
df_nv1 = df_nv1[(df_nv1['company_rank'].notnull())]

In [36]:
df_nv1.sort_values(by='company_rank',inplace = True)

In [37]:
df_nv1['company_rank'] = df_nv1.company_rank.astype(int)

In [38]:
df_nv2 = df_nv1.reset_index(drop=True)

In [40]:
#reindex the data as starting from 1
re_index_nv2 = pd.Series(range(1,277)) #<---- Edit the range for each year 
df_nv2 = df_nv2.set_index([re_index_nv2]) 

In [41]:
df_nv2

,BoY,Born,Education,ceo,ceo_year,company_rank,Nationality
1,1966.0,"October 17, 1966 (age 53 years), Memphis, TN","Bentonville High School, Sam M. Walton College of Business, The University of Tulsa, University of Arkansas",Doug McMillon,2018.0,1,United States
2,1964.0,"September 1964 (age 55 years), Shanghai, China",East China University of Science and Technology,Zhang Jianhua,2018.0,4,China
3,1958.0,"April 23, 1958 (age 62 years), Netherlands",Delft University of Technology (1981–1983),Ben van Beurden,2018.0,5,Netherlands
4,1956.0,"May 3, 1956 (age 63 years), Nagoya, Aichi, Japan","Babson College, Keio University",Akio Toyoda,2018.0,6,Japan
5,1958.0,"October 24, 1958 (age 61 years), Munich, Germany","Technische Universität München (1987), MORE",Herbert Diess,2018.0,7,Germany
6,1955.0,"September 14, 1955 (age 64 years), Queens, New York, NY","University of Illinois at Urbana-Champaign, MORE",Bob Dudley,2018.0,8,United States
7,NaN,"Wichita, KS","Kellogg School of Management, Texas A&M University",Darren Woods,2018.0,9,United States
8,1930.0,"August 30, 1930 (age 89 years), Omaha, NE","Columbia Business School (1950–1951), MORE",Warren Buffett,2018.0,10,United States
9,1960.0,"November 1, 1960 (age 59 years), Mobile, AL","The Fuqua School of Business (1988), MORE",Tim Cook,2018.0,11,United States
10,1959.0,"February 20, 1959 (age 61 years), Saint Paul, MN","Williams College of Business (1987), MORE",John H. Hammergren,2018.0,13,United States


In [42]:
df_nv2=df_nv2[["ceo", "company_rank", "Born", "Nationality", "Education", "BoY", "ceo_year"]]

In [43]:
df_nv2.to_csv(r'D:\Yeshiva_Univ\Courses\Capstone_project\CEOs_Culture_project\Data\ceo_information\ceo_bio\CEOs_info_2018_v1(Apr_25_2020).csv')